In [1]:
import argparse
import logging
import os
import random
import pandas as pd
from dataclasses import dataclass
from itertools import chain
from typing import Optional, Union
import csv
import math

import datasets
import torch
from datasets import load_dataset, load_metric, DatasetDict
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    PreTrainedTokenizerBase,
    default_data_collator,
    DataCollatorForSeq2Seq,
    AdamW,
    SchedulerType,
    get_scheduler,
    set_seed,
)
from transformers.file_utils import PaddingStrategy
from promptsource.templates import DatasetTemplates
pd.set_option("display.max_rows", 1200)


/opt/conda/envs/eds/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
torch.cuda.memory_reserved()

0

In [2]:
#Load model and tokenizer

model_name = "/home/transformers2"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Model and tokenizer loaded")

model.parallelize()
print("Moved model to GPUs")

Model and tokenizer loaded
Moved model to GPUs


In [4]:
for name, param in model.named_parameters():
    if name.startswith("encoder") or name.startswith("decoder"):
        param.detach()

In [5]:
# get all item_no and add as tokens
items = pd.read_parquet('data/item_no_6k.parquet.gzip')['item_no'].values.tolist()
tokenizer.add_tokens(items)

6036

In [6]:
padding = "max_length"
max_length = 1024
def tokenize_train(examples):
    input_texts = examples["input"]
    target_texts = examples["target"]

    model_inputs = tokenizer(
        input_texts,
        padding=padding,
        max_length=max_length,
        truncation=True,
    )

    with tokenizer.as_target_tokenizer():
        tokenized_targets = tokenizer(
            target_texts,
            padding=padding,
            max_length=max_length,
            truncation=True,
            add_special_tokens=False,
        )
        model_inputs["labels"] = [
            [(t if t != tokenizer.pad_token_id else -100) for t in targets]
            for targets in tokenized_targets["input_ids"]
        ]
    return model_inputs

  1%|          | 61/6181 [01:34<2:37:23,  1.54s/ba]


KeyboardInterrupt: 

In [18]:
d = DatasetDict.load_from_disk('data/tokenized')

In [17]:
d

DatasetDict({
    train: None
    test: Dataset({
        features: ['global_index', 'item_no', 'input', 'target', 'options', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 6180864
    })
})